## This Notebook is used for reproduction for the ONNX resuls (With Cuda 12.X)

In [1]:
# add path
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
print(sys.path)

Mounted at /content/drive
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Colab Notebooks']


In [2]:
pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [3]:
!pip install transformers accelerate auto-gptq autoawq onnx torchao optimum datasets==2.14.7 zstandard==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.7/417.7 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [5]:
import torch
from torchao.quantization import quant_api
from sklearn.metrics import accuracy_score
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
import time
from torch.nn.utils import prune
import pandas as pd
import torch
import onnx
import onnxruntime

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM

def onnx_convert_classification(model_load_method, tokenizer_load_method, sample_input, model_checkpoint=None, tokenizer=None, onnx_path=None, device="cpu", op_ver=14, architecture="bert"):
    """
    Converts a model to its ONNX representation.

    Parameters:
    - model_checkpoint: Identifier for the model. Can be a HuggingFace model ID, local path, or model in the environment.
    - tokenizer: Identifier for the tokenizer. Can be a HuggingFace tokenizer ID, local path, or tokenizer in the environment.
    - model_load_method: Method to load the model. Values: ['huggingface', 'local', 'env'].
    - tokenizer_load_method: Method to load the tokenizer. Values: ['huggingface', 'local', 'env'].
    - sample_input: Sample input for ONNX to set up the input format.
    - onnx_path: Path to store the ONNX model.
    - device: Execution device ('cpu' or 'cuda').
    - op_ver: ONNX opset version.
    - architecture: Model architecture ('bert' or 'gpt').
    """

    if model_load_method in ["huggingface", "local"]:
        if architecture == "bert":
            model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
        elif architecture == "gpt":
            model = AutoModelForCausalLM.from_pretrained(model_checkpoint).eval().to(device)

    if tokenizer_load_method in ["huggingface", "local"]:
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    if model is None or tokenizer is None:
        print("Error loading model or tokenizer")
        return None

    # Prepare inputs for ONNX export
    inputs = tokenizer(sample_input, return_tensors="pt", padding=True, truncation=True).to(device)
    input_args = (inputs['input_ids'],) if architecture == "gpt" else (inputs['input_ids'], inputs['attention_mask'])

    # Export to ONNX
    torch.onnx.export(model,
                      args=input_args,
                      f=onnx_path,
                      input_names=['input_ids'] if architecture == "gpt" else ['input_ids', 'attention_mask'],
                      output_names=['logits'],
                      dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                                    'attention_mask': {0: 'batch_size', 1: 'sequence_length'} if architecture != "gpt" else {},
                                    'logits': {0: 'batch_size'}},
                      opset_version=op_ver)

    return model


In [13]:

dataset = load_dataset("heegyu/augesc")
label_map = {
    "Question":0,
    "Restatement or Paraphrasing":1,
    "Reflection of feelings":2,
    "Self-disclosure":3,
    "Affirmation and Reassurance":4,
    "Providing Suggestions":5,
    "Information":6,
    "Others":7
}
x = []
y_true = []
for sample in dataset['test']:
    for row in sample['dialog']:
        text = row['text']
        label = row['strategy']
        if label != None:
            x.append(text)
            y_true.append(label_map[label])
x = x[0:1000]
y_true = y_true[0:1000]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# model = onnx_convert_classification(model_load_method = "hugginface",
#                             tokenizer_load_method = "hugginface",
#                             sample_input = x[0],
#                             model_checkpoint = "elisachen/gptq-tinyllama-classification",
#                             tokenizer = "heegyu/TinyLlama-augesc-context",
#                             onnx_path = "model.onnx",
#                             device = "cpu",
#                             op_ver = 14)

In [7]:
%env CUDA_LAUNCH_BLOCKING=1

import torch
import numpy as np
import onnxruntime as ort
import time
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

def evaluate_model_performance(model, tokenizer, x, y_true, device="cpu"):
    # start_time = time.time()
    # with torch.no_grad():
    #     outputs = model(**inputs)
    # end_time = time.time()
    # logits = outputs.logits
    # predictions = torch.argmax(logits, dim=-1)
    # correct_predictions = (predictions == target_label).sum().item()
    # accuracy = correct_predictions / predictions.shape[0]
    # inference_speed = end_time - start_time
    # return inference_speed, accuracy

    # calculate model size
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    model_size = (param_size + buffer_size) / 1024**2
    print('Base Model size: {:.3f}MB'.format(model_size))

    # inference part - measure speed and accuracy
    y_pred = []
    times = []
    for current_x, current_y in zip(x, y_true):
        inputs = tokenizer(current_x, return_tensors="pt").to(device)
        start_time = time.time()
        logits = model(**inputs).logits.softmax(-1)
        end_time = time.time()
        label = logits.argmax(-1).item()
        y_pred.append(label)
        times.append(end_time - start_time)
    acc = accuracy_score(y_true, y_pred)
    print(acc)
    print(pd.Series(times).describe().T)
    return model_size, sum(times) / len(times), acc



def evaluate_onnx_model(ort_session, tokenizer, texts, y_true, device = "cpu"):

    predictions = []
    times = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        onnx_inputs = {'input_ids': np.ascontiguousarray(inputs['input_ids'].cpu().numpy()),
                       'attention_mask': np.ascontiguousarray(inputs['attention_mask'].cpu().numpy())}


        start_time = time.time()
        logits = ort_session.run(None, onnx_inputs)[0]
        end_time = time.time()
        probs = softmax(logits)
        prediction = np.argmax(probs, axis=-1).item()
        predictions.append(prediction)
        times.append(end_time - start_time)
    predictions = np.array(predictions)
    acc = accuracy_score(y_true, predictions)
    inference_speed = sum(times) / len(times)
    print(acc)
    print(pd.Series(times).describe().T)
    return inference_speed, acc

def model_to_onnx_and_evaluate(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver, x, y):
    # Load the model and tokenizer according to the specified method
    if model_load_method == "huggingface":
        model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
    # Add conditions for 'local' or 'env' if needed

    if tokenizer_load_method == "huggingface":
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    # Add conditions for 'local' or 'env' if needed

    #inputs = #tokenizer(sample_input, return_tensors="pt", padding=True, truncation=True).to(device)
    #inputs = tokenizer(sample_input, return_tensors="pt").to(device)
    #target_label = y#torch.tensor([y], dtype=torch.long).to(device)  # Adjust based on your actual label format
    original_size, original_inference_speed, original_accuracy = evaluate_model_performance(model, tokenizer, x, y, device)
    print(f"Original Model - Size: {original_size} Inference Speed: {original_inference_speed}s, Accuracy: {original_accuracy}")
    start_time = time.time()
    model = onnx_convert_classification(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver = 14)
    end_time = time.time()
    print(f"ONNX converting time is {end_time - start_time}s")
  # Change to 'CUDAExecutionProvider' if using GPU
    # Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization.

    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    sess_options.optimized_model_filepath = onnx_path
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider'])
    # Evaluate the ONNX model
    #ort_session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider'])

    onnx_inference_speed, onnx_accuracy = evaluate_onnx_model(ort_session, tokenizer, x, y, device = device)
    print(f"ONNX Model - Inference Speed: {onnx_inference_speed}s, Accuracy: {onnx_accuracy}")

    return {
        "original": {"inference_speed": original_inference_speed, "accuracy": original_accuracy},
        "onnx": {"inference_speed": onnx_inference_speed, "accuracy": onnx_accuracy}
    }



env: CUDA_LAUNCH_BLOCKING=1


## Base Llama

In [ ]:
results = model_to_onnx_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            sample_input = x[0],
                            model_checkpoint = "heegyu/TinyLlama-augesc-context",
                            tokenizer_checkpoint = "heegyu/TinyLlama-augesc-context",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            x=x,
                            y=y_true
                            )

print("Evaluation Results:", results)

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Base Model size: 3972.417MB
0.222
count    1000.000000
mean        0.072956
std         0.035022
min         0.062838
25%         0.067868
50%         0.069584
75%         0.073242
max         1.158188
dtype: float64
Original Model - Size: 3972.416748046875 Inference Speed: 0.07295594453811645s, Accuracy: 0.222


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ONNX converting time is 59.94074821472168s
0.222
count    1000.000000
mean        0.049833
std         0.007614
min         0.043169
25%         0.047630
50%         0.048673
75%         0.050080
max         0.234887
dtype: float64
ONNX Model - Inference Speed: 0.049832817316055295s, Accuracy: 0.222
Evaluation Results: {'original': {'inference_speed': 0.07295594453811645, 'accuracy': 0.222}, 'onnx': {'inference_speed': 0.049832817316055295, 'accuracy': 0.222}}


In [ ]:
(0.04758974599838257 - 0.07280741047859192) / 0.07280741047859192

-0.3463612332102414

## GPTQ Llama

In [ ]:
results = model_to_onnx_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            sample_input = x[0],
                            model_checkpoint = "elisachen/gptq-tinyllama-classification",
                            tokenizer_checkpoint = "elisachen/gptq-tinyllama-classification",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            x=x,
                            y=y_true
                            )

print("Evaluation Results:", results)

config.json:   0%|          | 0.00/101k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/768M [00:00<?, ?B/s]

The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at elisachen/gptq-tinyllama-classification and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Base Model size: 622.512MB
0.075
count    1000.000000
mean        0.112215
std         0.033384
min         0.066217
25%         0.090390
50%         0.103346
75%         0.123388
max         0.316154
dtype: float64
Original Model - Size: 622.512451171875 Inference Speed: 0.1122153787612915s, Accuracy: 0.075


`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at elisachen/gptq-tinyllama-classification and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ONNX converting time is 46.52734327316284s
0.06
count    1000.000000
mean        0.031261
std         0.001203
min         0.028260
25%         0.030483
50%         0.031168
75%         0.031953
max         0.039784
dtype: float64
ONNX Model - Inference Speed: 0.03126149606704712s, Accuracy: 0.06
Evaluation Results: {'original': {'inference_speed': 0.1122153787612915, 'accuracy': 0.075}, 'onnx': {'inference_speed': 0.03126149606704712, 'accuracy': 0.06}}


In [ ]:
(0.03100110340118408 - 0.10758630299568177) / 0.10758630299568177

-0.7118489757713082

## AWQ Llama

In [ ]:
results = model_to_onnx_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            sample_input = x[0],
                            model_checkpoint = "elisachen/awq-tinyllama-classification",
                            tokenizer_checkpoint = "elisachen/awq-tinyllama-classification",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            x=x,
                            y=y_true
                            )

print("Evaluation Results:", results)

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/635M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Base Model size: 620.257MB
0.044
count    1000.000000
mean        0.073535
std         0.007162
min         0.061906
25%         0.067349
50%         0.073360
75%         0.076753
max         0.128123
dtype: float64
Original Model - Size: 620.256591796875 Inference Speed: 0.07353518533706666s, Accuracy: 0.044


`low_cpu_mem_usage` was None, now set to True since model is quantized.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ONNX converting time is 50.81122040748596s


RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Reshape node. Name:'/model/layers.0/self_attn/q_proj/Reshape' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/reshape_helper.h:45 onnxruntime::ReshapeHelper::ReshapeHelper(const onnxruntime::TensorShape&, onnxruntime::TensorShapeVector&, bool) input_shape_size == size was false. The input tensor cannot be reshaped to the requested shape. Input shape:{17,2048}, requested shape:{1,16,2048}


In [ ]:
0.0072 * 0.47

0.003384

# BERT

In [6]:
# fine-tune BERT model for our data
dataset_bert = load_dataset("sem_eval_2018_task_1", "subtask5.english")

labels = [label for label in dataset_bert['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Tweet"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# preprocess data
encoded_dataset = dataset_bert.map(preprocess_data, batched=True, remove_columns=dataset_bert['train'].column_names)
encoded_dataset.set_format("torch")

Map:   0%|          | 0/6838 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

Map:   0%|          | 0/886 [00:00<?, ? examples/s]

In [52]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 886
    })
})

In [44]:
encoded_dataset["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 6838
})

In [54]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

def convert_to_onnx_with_dataset(model_load_method, tokenizer_load_method, dataset, model_checkpoint=None, onnx_path=None, device="cpu", op_ver=14, architecture="bert"):
    """
    Converts a model to its ONNX representation, using a DatasetDict as input.

    Parameters:
    - dataset: A `DatasetDict` containing 'train', 'test', and 'validation' datasets.
    - model_checkpoint: Identifier for the model. Can be a HuggingFace model ID, local path, or model in the environment.
    - onnx_path: Path to store the ONNX model.
    - device: Execution device ('cpu' or 'cuda').
    - op_ver: ONNX opset version.
    - architecture: Model architecture ('bert' or 'gpt').
    """

    if model_load_method in ["huggingface", "local"]:
        if architecture == "bert":
            model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
        elif architecture == "gpt":
            model = AutoModelForCausalLM.from_pretrained(model_checkpoint).eval().to(device)

    if tokenizer_load_method in ["huggingface", "local"]:
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    if model is None or tokenizer is None:
        print("Error loading model or tokenizer")
        return None

    # Use a data collator to handle padding dynamically
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

    # Create a DataLoader for one of the splits (e.g., 'train')
    dataloader = DataLoader(dataset['train'], batch_size=1, collate_fn=data_collator)

    # Get a sample batch from the DataLoader
    sample_batch = next(iter(dataloader))
    sample_input = {k: v.to(device) for k, v in sample_batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}
    input_args = (sample_input['input_ids'],) if architecture == "gpt" else (sample_input['input_ids'], sample_input['attention_mask'])

    # Adjust dynamic axes based on model architecture
    dynamic_axes = {'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                    'logits': {0: 'batch_size'}}
    if architecture != "gpt":
        dynamic_axes.update({'attention_mask': {0: 'batch_size', 1: 'sequence_length'}})
    if 'token_type_ids' in sample_input:
        dynamic_axes.update({'token_type_ids': {0: 'batch_size', 1: 'sequence_length'}})

    # Export to ONNX
    torch.onnx.export(model,
                      args=input_args,
                      f=onnx_path,
                      input_names=list(sample_input.keys()),
                      output_names=['logits'],
                      dynamic_axes=dynamic_axes,
                      opset_version=op_ver)

    return model


def evaluate_model_performance(model, dataloader, device="cpu"):
    """Evaluates the model's performance on a given DataLoader."""
    model.eval()
    y_true = []
    y_pred = []
    times = []
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            start_time = time.time()
            outputs = model(**inputs)
            logits = outputs.logits.squeeze()
            end_time = time.time()

            # logits = outputs.logits
            # predictions = torch.sigmoid(logits)
            # probs = np.zeros(predictions.shape)
            # probs[np.where(predictions.cpu().numpy() >= 0.5)] = 1
            predictions.append(logits)
            y_true.append(labels.cpu().numpy())
            times.append(end_time - start_time)
    # sigmoid = torch.nn.Sigmoid()
    # probs = sigmoid(torch.Tensor(predictions))
    # y_pred = np.zeros(probs.shape)
    # y_pred[np.where(probs >= 0.5)] = 1
    # acc = accuracy_score(y_true, y_pred)
    inference_speed = np.mean(times)
    #print(f"Model Performance - Accuracy: {"none"}, Average Inference Time: {inference_speed}s")

    # Calculate model size in MB
    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad) * 4 / 1024**2
    print(f'Base Model size: {model_size:.3f}MB')
    print(pd.Series(times).describe().T)
    return model_size, inference_speed#, acc

def evaluate_onnx_model(ort_session, dataloader):
    """Evaluates the ONNX model's performance."""
    y_true = []
    y_pred = []
    times = []

    for batch in dataloader:
        input_ids = batch['input_ids'].numpy()
        attention_mask = batch['attention_mask'].numpy()
        token_type_ids = batch['token_type_ids'].numpy()
        labels = batch['labels'].numpy()

        onnx_inputs = {'input_ids': input_ids, 'token_type_ids':token_type_ids} #'attention_mask': attention_mask,
        start_time = time.time()
        logits = ort_session.run(None, onnx_inputs)[0]
        end_time = time.time()
        # probs = torch.sigmoid(logits)
        # predictions = np.zeros(probs.shape)
        # predictions[np.where(probs.cpu().numpy() >= 0.5)] = 1
        # y_pred.append(predictions)
        # y_true.append(labels)
        times.append(end_time - start_time)
    #acc = accuracy_score(y_true, y_pred)
    inference_speed = np.mean(times)
    print(pd.Series(times).describe().T)
    print(f"ONNX Model Performance - Accuracy: , Average Inference Time: {inference_speed}s")

    return inference_speed#, acc

def model_to_onnx_and_evaluate(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver, x, y):
    # Load the model and tokenizer according to the specified method
    if model_load_method == "huggingface":
        model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
    # Add conditions for 'local' or 'env' if needed

    if tokenizer_load_method == "huggingface":
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    # Add conditions for 'local' or 'env' if needed

    #inputs = #tokenizer(sample_input, return_tensors="pt", padding=True, truncation=True).to(device)
    #inputs = tokenizer(sample_input, return_tensors="pt").to(device)
    #target_label = y#torch.tensor([y], dtype=torch.long).to(device)  # Adjust based on your actual label format
    original_inference_speed= evaluate_model_performance(model, tokenizer, x, y, device)
    #print(f"Original Model - Size: {original_size} Inference Speed: {original_inference_speed}s, Accuracy: ")

    model = onnx_convert_classification(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver = 14)

  # Change to 'CUDAExecutionProvider' if using GPU
    # Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization.

    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    sess_options.optimized_model_filepath = onnx_path
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider'])
    # Evaluate the ONNX model
    #ort_session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider'])

    onnx_inference_speed= evaluate_onnx_model(ort_session, tokenizer, x, y, device = device)
    print(f"ONNX Model - Inference Speed: {onnx_inference_speed}s, Accuracy: ")

    return {
        "original": {"inference_speed": original_inference_speed, "accuracy": "dummy"},
        "onnx": {"inference_speed": onnx_inference_speed, "accuracy": "dummy"}
    }


import os
import torch
import onnxruntime as ort
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

def onnx_convert_and_evaluate(model_load_method, tokenizer_load_method, model_checkpoint, tokenizer_checkpoint, dataset_dict, onnx_path, device="cpu", op_ver=14, batch_size=1, architecture = "bert"):
    """
    Convert a Hugging Face model to ONNX format and evaluate performance on a given dataset.

    Parameters:
    - model_checkpoint: The path or identifier for the pretrained model.
    - tokenizer_checkpoint: The path or identifier for the tokenizer.
    - dataset_dict: A `DatasetDict` containing 'train', 'test', and 'validation' splits.
    - onnx_path: The file path to save the converted ONNX model.
    - device: The device to perform the conversion and evaluation on.
    - op_ver: ONNX opset version.
    - batch_size: The size of the batch for evaluation.
    """
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

    # Prepare DataLoader for evaluation
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
    dataloader = DataLoader(dataset_dict['train'], batch_size=batch_size, collate_fn=data_collator)

    # Load the model and evaluate its performance before conversion
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to(device)
    original_inference_speed= evaluate_model_performance(model, dataloader, device=device)
    print(f"Original Model - Size: MB, Inference Speed: {original_inference_speed}s, Accuracy:")

    # Convert the model to ONNX
    # Assuming `onnx_convert_classification` is defined and implements the conversion
    start_time = time.time()
    model = convert_to_onnx_with_dataset(model_load_method, tokenizer_load_method, dataset_dict, model_checkpoint, onnx_path, device, op_ver, architecture)
    end_time = time.time()
    print(f"ONNX converting time is {end_time - start_time}s")
    # Set up ONNX Runtime session
    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider' if device == "cuda" else 'CPUExecutionProvider'])

    # Evaluate the ONNX model
    onnx_inference_speed= evaluate_onnx_model(ort_session, dataloader)
    print(f"ONNX Model - Inference Speed: {onnx_inference_speed}s, Accuracy:")

    return {
        "original": {"inference_speed_s": original_inference_speed},
        "onnx": {"inference_speed_s": onnx_inference_speed}
    }


results = onnx_convert_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            dataset_dict = encoded_dataset,
                            model_checkpoint = "adjohn1313/bert-base-finetuned",
                            tokenizer_checkpoint = "adjohn1313/bert-base-finetuned",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            architecture = "bert"
                            )

print("Evaluation Results:", results)

Base Model size: 417.674MB
count    6838.000000
mean        0.008881
std         0.000456
min         0.008125
25%         0.008608
50%         0.008771
75%         0.009020
max         0.013337
dtype: float64
Original Model - Size: MB, Inference Speed: (417.6738700866699, 0.008880815543637495)s, Accuracy:
ONNX converting time is 5.764698505401611s
count    6838.000000
mean        0.008012
std         0.000342
min         0.007556
25%         0.007939
50%         0.007982
75%         0.008104
max         0.017314
dtype: float64
ONNX Model Performance - Accuracy: , Average Inference Time: 0.008012400764092677s
ONNX Model - Inference Speed: 0.008012400764092677s, Accuracy:
Evaluation Results: {'original': {'inference_speed_s': (417.6738700866699, 0.008880815543637495)}, 'onnx': {'inference_speed_s': 0.008012400764092677}}


In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

def convert_to_onnx_with_dataset(model_load_method, tokenizer_load_method, dataset, model_checkpoint=None, onnx_path=None, device="cpu", op_ver=14, architecture="bert"):
    """
    Converts a model to its ONNX representation, using a DatasetDict as input.

    Parameters:
    - dataset: A `DatasetDict` containing 'train', 'test', and 'validation' datasets.
    - model_checkpoint: Identifier for the model. Can be a HuggingFace model ID, local path, or model in the environment.
    - onnx_path: Path to store the ONNX model.
    - device: Execution device ('cpu' or 'cuda').
    - op_ver: ONNX opset version.
    - architecture: Model architecture ('bert' or 'gpt').
    """

    if model_load_method in ["huggingface", "local"]:
        if architecture == "bert":
            model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
        elif architecture == "gpt":
            model = AutoModelForCausalLM.from_pretrained(model_checkpoint).eval().to(device)

    if tokenizer_load_method in ["huggingface", "local"]:
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    if model is None or tokenizer is None:
        print("Error loading model or tokenizer")
        return None

    # Use a data collator to handle padding dynamically
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

    # Create a DataLoader for one of the splits (e.g., 'train')
    dataloader = DataLoader(dataset['validation'], batch_size=1, collate_fn=data_collator)

    # Get a sample batch from the DataLoader
    sample_batch = next(iter(dataloader))
    sample_input = {k: v.to(device) for k, v in sample_batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}
    input_args = (sample_input['input_ids'],) if architecture == "gpt" else (sample_input['input_ids'], sample_input['attention_mask'])

    # Adjust dynamic axes based on model architecture
    dynamic_axes = {'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                    'logits': {0: 'batch_size'}}
    if architecture != "gpt":
        dynamic_axes.update({'attention_mask': {0: 'batch_size', 1: 'sequence_length'}})
    if 'token_type_ids' in sample_input:
        dynamic_axes.update({'token_type_ids': {0: 'batch_size', 1: 'sequence_length'}})

    # Export to ONNX
    torch.onnx.export(model,
                      args=input_args,
                      f=onnx_path,
                      input_names=list(sample_input.keys()),
                      output_names=['logits'],
                      dynamic_axes=dynamic_axes,
                      opset_version=op_ver)

    return model


def evaluate_model_performance(model, dataloader, device="cpu"):
    """Evaluates the model's performance on a given DataLoader."""
    model.eval()
    y_true = []
    y_pred = []
    times = []
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels' and k != "token_type_ids"}
            labels = batch['labels'].to(device)
            start_time = time.time()
            outputs = model(**inputs)
            logits = outputs.logits.squeeze()
            end_time = time.time()

            # logits = outputs.logits
            # predictions = torch.sigmoid(logits)
            # probs = np.zeros(predictions.shape)
            # probs[np.where(predictions.cpu().numpy() >= 0.5)] = 1
            predictions.append(logits)
            y_true.append(labels.cpu().numpy())
            times.append(end_time - start_time)
    # sigmoid = torch.nn.Sigmoid()
    # probs = sigmoid(torch.Tensor(predictions))
    # y_pred = np.zeros(probs.shape)
    # y_pred[np.where(probs >= 0.5)] = 1
    # acc = accuracy_score(y_true, y_pred)
    print(pd.Series(times).describe().T)
    inference_speed = np.mean(times)
    #print(f"Model Performance - Accuracy: {"none"}, Average Inference Time: {inference_speed}s")

    # Calculate model size in MB
    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad) * 4 / 1024**2
    print(f'Base Model size: {model_size:.3f}MB')

    return model_size, inference_speed#, acc

def evaluate_onnx_model(ort_session, dataloader):
    """Evaluates the ONNX model's performance."""
    y_true = []
    y_pred = []
    times = []

    for batch in dataloader:
        input_ids = batch['input_ids'].numpy()
        attention_mask = batch['attention_mask'].numpy()
        token_type_ids = batch['token_type_ids'].numpy()
        labels = batch['labels'].numpy()

        onnx_inputs = {'input_ids': input_ids, 'token_type_ids':token_type_ids} #'attention_mask': attention_mask,
        start_time = time.time()
        logits = ort_session.run(None, onnx_inputs)[0]
        end_time = time.time()
        # probs = torch.sigmoid(logits)
        # predictions = np.zeros(probs.shape)
        # predictions[np.where(probs.cpu().numpy() >= 0.5)] = 1
        # y_pred.append(predictions)
        # y_true.append(labels)
        times.append(end_time - start_time)
    #acc = accuracy_score(y_true, y_pred)
    print(pd.Series(times).describe().T)
    inference_speed = np.mean(times)
    print(f"ONNX Model Performance - Accuracy: , Average Inference Time: {inference_speed}s")

    return inference_speed#, acc

def model_to_onnx_and_evaluate(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver, x, y):
    # Load the model and tokenizer according to the specified method
    if model_load_method == "huggingface":
        model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).eval().to(device)
    # Add conditions for 'local' or 'env' if needed

    if tokenizer_load_method == "huggingface":
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    # Add conditions for 'local' or 'env' if needed

    #inputs = #tokenizer(sample_input, return_tensors="pt", padding=True, truncation=True).to(device)
    #inputs = tokenizer(sample_input, return_tensors="pt").to(device)
    #target_label = y#torch.tensor([y], dtype=torch.long).to(device)  # Adjust based on your actual label format
    original_inference_speed= evaluate_model_performance(model, tokenizer, x, y, device)
    #print(f"Original Model - Size: {original_size} Inference Speed: {original_inference_speed}s, Accuracy: ")

    model = onnx_convert_classification(model_load_method, tokenizer_load_method, sample_input, model_checkpoint, tokenizer_checkpoint, onnx_path, device, op_ver = 14)

  # Change to 'CUDAExecutionProvider' if using GPU
    # Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization.

    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    sess_options.optimized_model_filepath = onnx_path
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider'])
    # Evaluate the ONNX model
    #ort_session = ort.InferenceSession(onnx_path, providers=['CUDAExecutionProvider'])

    onnx_inference_speed= evaluate_onnx_model(ort_session, tokenizer, x, y, device = device)
    print(f"ONNX Model - Inference Speed: {onnx_inference_speed}s, Accuracy: ")

    return {
        "original": {"inference_speed": original_inference_speed, "accuracy": "dummy"},
        "onnx": {"inference_speed": onnx_inference_speed, "accuracy": "dummy"}
    }


import os
import torch
import onnxruntime as ort
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

def onnx_convert_and_evaluate(model_load_method, tokenizer_load_method, model_checkpoint, tokenizer_checkpoint, dataset_dict, onnx_path, device="cpu", op_ver=14, batch_size=1, architecture = "bert"):
    """
    Convert a Hugging Face model to ONNX format and evaluate performance on a given dataset.

    Parameters:
    - model_checkpoint: The path or identifier for the pretrained model.
    - tokenizer_checkpoint: The path or identifier for the tokenizer.
    - dataset_dict: A `DatasetDict` containing 'train', 'test', and 'validation' splits.
    - onnx_path: The file path to save the converted ONNX model.
    - device: The device to perform the conversion and evaluation on.
    - op_ver: ONNX opset version.
    - batch_size: The size of the batch for evaluation.
    """
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

    # Prepare DataLoader for evaluation
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
    dataloader = DataLoader(dataset_dict['validation'], batch_size=batch_size, collate_fn=data_collator)

    # Load the model and evaluate its performance before conversion
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to(device)
    original_inference_speed= evaluate_model_performance(model, dataloader, device=device)
    print(f"Original Model - Size: MB, Inference Speed: {original_inference_speed}s, Accuracy:")

    # Convert the model to ONNX
    # Assuming `onnx_convert_classification` is defined and implements the conversion
    start_time = time.time()
    model = convert_to_onnx_with_dataset(model_load_method, tokenizer_load_method, dataset_dict, model_checkpoint, onnx_path, device, op_ver, architecture)
    end_time = time.time()
    print(f"ONNX converting time is {end_time - start_time}s")
    # Set up ONNX Runtime session
    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider' if device == "cuda" else 'CPUExecutionProvider'])

    # Evaluate the ONNX model
    onnx_inference_speed= evaluate_onnx_model(ort_session, dataloader)
    print(f"ONNX Model - Inference Speed: {onnx_inference_speed}s, Accuracy:")

    return {
        "original": {"inference_speed_s": original_inference_speed},
        "onnx": {"inference_speed_s": onnx_inference_speed}
    }


results = onnx_convert_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            dataset_dict = encoded_dataset,
                            model_checkpoint = "adjohn1313/bert-base-finetuned",
                            tokenizer_checkpoint = "adjohn1313/bert-base-finetuned",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            architecture = "bert"
                            )

print("Evaluation Results:", results)

count    886.000000
mean       0.008952
std        0.000443
min        0.008230
25%        0.008672
50%        0.008895
75%        0.009144
max        0.012131
dtype: float64
Base Model size: 417.674MB
Original Model - Size: MB, Inference Speed: (417.6738700866699, 0.008951938986509164)s, Accuracy:
ONNX converting time is 6.173702716827393s
count    886.000000
mean       0.007688
std        0.000943
min        0.007392
25%        0.007498
50%        0.007566
75%        0.007640
max        0.017392
dtype: float64
ONNX Model Performance - Accuracy: , Average Inference Time: 0.00768828607305178s
ONNX Model - Inference Speed: 0.00768828607305178s, Accuracy:
Evaluation Results: {'original': {'inference_speed_s': (417.6738700866699, 0.008951938986509164)}, 'onnx': {'inference_speed_s': 0.00768828607305178}}


In [47]:
results = onnx_convert_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            dataset_dict = encoded_dataset,
                            model_checkpoint = "Jennny/pruned_distillbert",
                            tokenizer_checkpoint = "Jennny/pruned_distillbert",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            architecture = "bert"
                            )

print("Evaluation Results:", results)

count    886.000000
mean       0.005312
std        0.002078
min        0.004574
25%        0.004896
50%        0.005066
75%        0.005275
max        0.063236
dtype: float64
Base Model size: 255.439MB
Original Model - Size: MB, Inference Speed: (255.43949508666992, 0.005312252529049565)s, Accuracy:
ONNX converting time is 3.5583243370056152s
count    886.000000
mean       0.003952
std        0.000936
min        0.003702
25%        0.003771
50%        0.003795
75%        0.003817
max        0.020726
dtype: float64
ONNX Model Performance - Accuracy: , Average Inference Time: 0.003951605501885338s
ONNX Model - Inference Speed: 0.003951605501885338s, Accuracy:
Evaluation Results: {'original': {'inference_speed_s': (255.43949508666992, 0.005312252529049565)}, 'onnx': {'inference_speed_s': 0.003951605501885338}}


In [26]:
results = onnx_convert_and_evaluate(
                            model_load_method = "huggingface",
                            tokenizer_load_method = "huggingface",
                            dataset_dict = encoded_dataset,
                            model_checkpoint = "Jennny/pruned_distillbert",
                            tokenizer_checkpoint = "Jennny/pruned_distillbert",
                            onnx_path = "model.onnx",
                            device = "cuda",
                            op_ver = 14,
                            architecture = "bert"
                            )

print("Evaluation Results:", results)

TypeError: DistilBertForSequenceClassification.forward() got an unexpected keyword argument 'token_type_ids'

In [9]:
import os
import torch
import onnxruntime as ort
import numpy as np
import pandas as pd
import time
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

def evaluate_model_performance(model, dataloader, device="cpu"):
    """Evaluates the model's performance on a given DataLoader."""
    model.eval()
    times = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            start_time = time.time()
            outputs = model(**inputs)
            end_time = time.time()
            times.append(end_time - start_time)

    inference_speed = np.mean(times)
    print(f'Model Performance - Average Inference Time: {inference_speed}s')
    return inference_speed

def evaluate_onnx_model(ort_session, dataloader):
    """Evaluates the ONNX model's performance."""
    times = []

    for batch in dataloader:
        input_ids = batch['input_ids'].numpy()
        attention_mask = batch['attention_mask'].numpy()
        labels = batch['labels'].numpy()

        onnx_inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        start_time = time.time()
        logits = ort_session.run(None, onnx_inputs)[0]
        end_time = time.time()
        times.append(end_time - start_time)

    inference_speed = np.mean(times)
    print(f"ONNX Model Performance - Average Inference Time: {inference_speed}s")
    return inference_speed

def onnx_convert_and_evaluate(model_load_method, tokenizer_load_method, dataset_dict, model_checkpoint, tokenizer_checkpoint, onnx_path, device="cpu", op_ver=14, batch_size=8, architecture="bert"):
    """
    Convert a Hugging Face model to ONNX format and evaluate performance on a given dataset.
    """
    # Load the tokenizer and prepare DataLoader
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
    dataloader = DataLoader(dataset_dict['train'], batch_size=batch_size, collate_fn=data_collator)

    # Load the model and evaluate its performance before conversion
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to(device)
    original_inference_speed = evaluate_model_performance(model, dataloader, device=device)

    # Convert the model to ONNX and set up ONNX Runtime session
    torch.onnx.export(model,
                      args=(next(iter(dataloader))['input_ids'].to(device), next(iter(dataloader))['attention_mask'].to(device)),
                      f=onnx_path,
                      input_names=['input_ids', 'attention_mask'],
                      output_names=['logits'],
                      dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                                    'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                                    'logits': {0: 'batch_size'}},
                      opset_version=op_ver)
    sess_options = ort.SessionOptions()
    sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    ort_session = ort.InferenceSession(onnx_path, sess_options, providers=['CUDAExecutionProvider' if device == "cuda" else 'CPUExecutionProvider'])

    # Evaluate the ONNX model
    onnx_inference_speed = evaluate_onnx_model(ort_session, dataloader)

    return {
        "original": {"inference_speed_s": original_inference_speed},
        "onnx": {"inference_speed_s": onnx_inference_speed}
    }

# Example usage
results = onnx_convert_and_evaluate(
    model_load_method = "huggingface",
    tokenizer_load_method = "huggingface",
    dataset_dict = encoded_dataset,  # Replace 'encoded_dataset' with your actual DatasetDict
    model_checkpoint = "adjohn1313/bert-base-finetuned",
    tokenizer_checkpoint = "adjohn1313/bert-base-finetuned",
    onnx_path = "model.onnx",
    device = "cuda",
    op_ver = 14,
    architecture = "bert"
)

print("Evaluation Results:", results)


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Model Performance - Average Inference Time: 0.010821763534992063s
ONNX Model Performance - Average Inference Time: 0.017326509743406063s
Evaluation Results: {'original': {'inference_speed_s': 0.010821763534992063}, 'onnx': {'inference_speed_s': 0.017326509743406063}}
